In [ ]:
# Create a world box
world = mt.createCube(size=[600, 40, 35], pos=[0, -20, 7.5])

# Create cylinders
cylinders = []
positions = [-20, -15, -10, -5, 0, 5, 10, 15]
for pos in positions:
    cylinder = mt.createCylinder(pos=[pos, 0, 0], radius=0.35, height=0.9, nSegments=32)
    cylinders.append(cylinder)

# Merge all geometries
geom = world
for cyl in cylinders:
    geom = geom + cyl
pg.show(geom, boundaryMarker=True)

In [ ]:
# Create terrain rectangles

for pos in positions:
    terra = mt.createRectangle(start=[pos + 0.35, 0], end=[pos + 9.65, -0.9])
    geom = geom + terra

terraNeg = mt.createRectangle(start=[-30, 0], end=[-19.65, -0.9])
terraPos = mt.createRectangle(start=[24.65, 0], end=[30, -0.9])
geom = geom + terraNeg + terraPos

In [ ]:
#tentativa de adicionar marcadores de região
    """

    #geom.addRegionMarker(top_boundaries, 3)
    #geom.addRegionMarker(bottom_boundaries, 4)
    #geom.addRegionMarker(left_boundaries, 5)
    #geom.addRegionMarker(right_boundaries, 6)
    #geom.addRegionMarker(side_boundaries, 7)
    #geom.addRegionMarker(world_boundaries, -2)

    geom.addRegionMarker(pos(bottom_boundaries[0]), 2)
    geom.addRegionMarker(pos(top_boundaries[0]), 0)
    geom.addRegionMarker(pos(side_boundaries[0]), 3)
    geom.addRegionMarker(pos(zero_boundaries[0]), -2) """


#tentativa de criar marcadores
    i = 0
    nodes = [pg.core.Node(coord[1], coord[2]) for coord in coord]
    line_segments = []

    for a in range(len(nodes) - 1):
        start_node = nodes[a]
        end_node = nodes[a + 1]

        if i == 0:
            marker = i + 1
            i += 1
        elif i == 1:
            marker = i + 2
            i += 1
        elif i == 2:
            marker = i + 3
            i += 1
        elif i == 3:
            marker = i + 4
            i = 0

        line_segment = mt.createLine(start=start_node.pos(), end=end_node.pos(), marker=marker)
        line_segments.append(line_segment)

    print('line_segments: ' + str(line_segments))
    pg.show( geom, label='Line segments', markers=True, figsize=(15, 8), orientation='vertical' )


In [ ]:
print(bottom_boundaries[0])

In [ ]:
#Tentativa Calculate the advection-dispersion term with dependency on height (y)
#Tentativa de advection
    for boundary_id in zero_boundaries:
        grad_c = np.gradient(p)  # Gradient of hydraulic potential
        div_grad_c = np.gradient(grad_c) + np.gradient(grad_c)  # Divergence of the gradient
        y = mesh.boundary(boundary_id).center()[1]  # Get the y-coordinate of the boundary center
        if len(grad_c) > 1 and boundary_id < len(grad_c[0]):  # Ensure grad_c has enough elements and boundary_id is within range
            #advection_dispersion = phi * p / dt - phi * D * div_grad_c - K * grad_c[0][boundary_id] - y
            #pBound["Robin"][boundary_id] = advection_dispersion
#tentativa
    for boundary_id in zero_boundaries:  # Calculate the advection-dispersion term
        grad_c = np.gradient(p)  # Gradient of hydraulic potential
        div_grad_c = np.gradient(grad_c) + np.gradient(grad_c)  # Divergence of the gradient
        if len(grad_c) > 1 and boundary_id < len(grad_c[0]):  # Ensure grad_c has enough elements and boundary_id is within range
            advection_dispersion = phi * p / dt - phi * D * div_grad_c - K * grad_c[0][boundary_id]
            pBound["Robin"][boundary_id] = advection_dispersion

#Tentativa h2o2
    for boundary_id in side_boundaries:
        boundary = mesh.boundary(boundary_id)
        if boundary is not None:
            y = boundary.center()[1]  # Get the y-coordinate of the boundary center
            h2o2 = h2o - y  # Calculate the water height at the boundary
            pBound["Neumann"][boundary_id] = h2o2  # Atualize a altura da água


# condiçoes de fronteira erradas
    for boundary_id in top_boundaries:
        pBound["Dirichlet"][boundary_id] = 0


    for boundary_id in bottom_boundaries:
        pBound["Neumann"][boundary_id] = h2o  # Atualize a altura da água

    # Debug print to check pBound contents
    #print("pBound:", pBound)

In [ ]:
#pg.show(geom, markers=True, nodes=True, boundaries=True)
for b in mesh.boundaries():
    if b.marker() == 0 and not b.outside():        # Se a fronteira é um furo
        b.setMarker(5)
    if b.marker() !=5:
        print(f"Boundary ID: {b.id()}, Marker: {b.marker()}")

# condicoes de fronteira corretas
for b in mesh.boundaries():
    if b.marker() == -2:                            # Se a fronteira é o mundo inferior
        pBound["Dirichlet"][b.id()] = 1

    if b.marker() == 1:                             # Se a fronteira é superior
        pBound["Dirichlet"][b.id()] = 0

    #print(mesh)
    #ax, _ = pg.show(mesh, markers=True, showMesh=True)                         # Mostrar a malha



# Debug print to check pBound contents
if -2 in pBound.get("Dirichlet", {}):
    print("pBound:", pBound)


In [ ]:

# Debug print to check pBound contents
print("pBound:", pBound)

In [ ]:
# Teste de visualização ERT 
# Assuming DD12 corresponds to a depth of -12 (replace with the actual depth value)
depth_threshold = -12

# Initialize ERTManager
ERT = ERTManager(verbose=False)

# Run simulation for the apparent resistivities
rhoa = ERT.simulate(meshERT, res=resis, scheme=ertScheme, returnArray=True, verbose=False)

# Filter the data below the depth threshold
filtered_rhoa = []
for i in range(len(rhoa)):
    mask = [ertScheme.sensorPosition(ertScheme('a')[j])[1] >= depth_threshold and
            ertScheme.sensorPosition(ertScheme('b')[j])[1] >= depth_threshold and
            ertScheme.sensorPosition(ertScheme('m')[j])[1] >= depth_threshold and
            ertScheme.sensorPosition(ertScheme('n')[j])[1] >= depth_threshold
            for j in range(ertScheme.size())]
    filtered_rhoa.append(np.array(rhoa[i])[mask])

# Visualize the filtered data
for i in range(4):
    ERT.showData(ertScheme, vals=filtered_rhoa[i], cMin=1e-4, cMax=1,)

In [ ]:
# Filter measurements based on a depth threshold
depth_threshold = -12  # Define depth cutoff at -12m

ERT = ERTManager(verbose=False)

# Run simulation for the apparent resistivities
rhoa = ERT.simulate(meshERT, res=resis, scheme=ertScheme, returnArray=True, verbose=False)

for i in range(4):
    # Create a mask for measurements above the depth threshold
    mask = [ertScheme.sensorPosition(ertScheme('a')[j])[1] >= depth_threshold and
           ertScheme.sensorPosition(ertScheme('b')[j])[1] >= depth_threshold and
           ertScheme.sensorPosition(ertScheme('m')[j])[1] >= depth_threshold and
           ertScheme.sensorPosition(ertScheme('n')[j])[1] >= depth_threshold
           for j in range(ertScheme.size())]
    
    # Apply the mask to the data
    filtered_data = rhoa[i][mask]/rhoa[0][mask]
    
    # Create a new scheme with only the filtered measurements
    filtered_scheme = ertScheme.filter(mask)
    
    # Show the filtered data
    ERT.showData(filtered_scheme, vals=filtered_data, logScale=False, 
                 label='Apparent resistivity (filtered)')

In [ ]:
ERT = ERTManager(verbose=False)

# Run simulation for the apparent resistivities
d = ERT.simulate(meshERT, res=resis, scheme=ertScheme, 
                    returnArray=True, verbose=False, )#noiseLevel=1, noiseAbs=1e-3,)

# Filter data to include only values with Y less than 12
mask = np.array([pg.y(ertScheme.sensorPosition(ertScheme('a')[i])) < 12 and
                 pg.y(ertScheme.sensorPosition(ertScheme('b')[i])) < 12 and
                 pg.y(ertScheme.sensorPosition(ertScheme('m')[i])) < 12 and
                 pg.y(ertScheme.sensorPosition(ertScheme('n')[i])) < 12
                 for i in range(ertScheme.size())])
filtered_data = d[:, mask]

# Create a new mesh from the filtered cells
filtered_cells = [cell for cell, m in zip(meshERT.cells(), mask) if m]
filtered_mesh = pg.Mesh(dim=2)
nodes = []
for cell in filtered_cells:
    for node in cell.nodes():
        if node not in nodes:
            nodes.append(node)
            filtered_mesh.createNode(node.pos())
    filtered_mesh.createCell([filtered_mesh.findNearestNode(n.pos()) for n in cell.nodes()])
# Assign the filtered data to the new mesh
filtered_mesh['d'] = filtered_data

#pg.show(filtered_mesh, figsize=(6,4), data=filtered_data,orientation='vertical')          

                    
# Plot the first 4 timesteps normalized by the first timestep
for i in range(4):
    # Create a copy of the scheme for each visualization
    tempScheme = ertScheme.copy()
    tempScheme.set('rhoa', filtered_data[i])
    ERT.showData(tempScheme, vals=filtered_data[i]/filtered_data[0], cMin=1e-4, cMax=1)